In [1]:
import os
import sys
import time
import pandas as pd
import numpy as np
from os import listdir
from generate_epoch import * # make sure to have this file inside the ML folder
from scipy.signal import butter, sosfiltfilt, sosfreqz  # for filtering
from neurodsp import filt
import random
import biosig
import json
from mne.channels import make_standard_montage
from mne.io import concatenate_raws, read_raw_gdf
# for models:
from pyriemann.estimation import XdawnCovariances
from pyriemann.tangentspace import TangentSpace
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from sklearn.metrics import classification_report
import pyeeg
from sklearn.metrics import accuracy_score

In [2]:
sample_data_folder = 'BCICIV_2b_gdf'
sample_data_raw_files = os.listdir(sample_data_folder)
sample_data_raw_files = [sample_data_folder + '/' + i for i in sample_data_raw_files if '.gdf' in i] 
sample_data_raw_files_train = [i for i in sample_data_raw_files if 'T' in i] 
sample_data_raw_files_test = [i for i in sample_data_raw_files if 'E' in i]
sample_data_raw_files_train.sort()
sample_data_raw_files_test.sort()

In [3]:
raws_train = [read_raw_gdf(f, preload=True) for f in sample_data_raw_files_train]
raws_test = [read_raw_gdf(f, preload=True) for f in sample_data_raw_files_test]
headers_train = [json.loads(biosig.header(f)) for f in sample_data_raw_files_train]
headers_test = [json.loads(biosig.header(f)) for f in sample_data_raw_files_test]
for r in raws_train:
    r.set_channel_types({'EEG:C3':'eeg', 'EEG:Cz':'eeg', 'EEG:C4':'eeg', 'EOG:ch01':'eog', 'EOG:ch02': 'eog',\
                      'EOG:ch03':'eog'})
    r.rename_channels({'EEG:C3':'C3', 'EEG:Cz':'Cz', 'EEG:C4':'C4'})
    ten_twenty_montage = make_standard_montage('standard_1020')
    r.set_montage(ten_twenty_montage)
for r in raws_test:
    r.set_channel_types({'EEG:C3':'eeg', 'EEG:Cz':'eeg', 'EEG:C4':'eeg', 'EOG:ch01':'eog', 'EOG:ch02': 'eog',\
                      'EOG:ch03':'eog'})
    r.rename_channels({'EEG:C3':'C3', 'EEG:Cz':'Cz', 'EEG:C4':'C4'})
    ten_twenty_montage = make_standard_montage('standard_1020')
    r.set_montage(ten_twenty_montage)
dfs_train = [r.to_data_frame() for r in raws_train]
dfs_test = [r.to_data_frame() for r in raws_test]

Extracting EDF parameters from /Users/simonfei/Motor-Imagery-App-Project/ML/BCICIV_2b_gdf/B0101T.gdf...
GDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 604802  =      0.000 ...  2419.208 secs...


/Users/simonfei/opt/anaconda3/lib/python3.7/site-packages/mne/io/edf/edf.py:1000: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  etmode = np.fromstring(etmode, np.uint8).tolist()[0]


Extracting EDF parameters from /Users/simonfei/Motor-Imagery-App-Project/ML/BCICIV_2b_gdf/B0102T.gdf...
GDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 552451  =      0.000 ...  2209.804 secs...


/Users/simonfei/opt/anaconda3/lib/python3.7/site-packages/mne/io/edf/edf.py:1000: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  etmode = np.fromstring(etmode, np.uint8).tolist()[0]


Extracting EDF parameters from /Users/simonfei/Motor-Imagery-App-Project/ML/BCICIV_2b_gdf/B0103T.gdf...
GDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 469010  =      0.000 ...  1876.040 secs...


...

/Users/simonfei/opt/anaconda3/lib/python3.7/site-packages/mne/io/edf/edf.py:1000: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  etmode = np.fromstring(etmode, np.uint8).tolist()[0]


Extracting EDF parameters from /Users/simonfei/Motor-Imagery-App-Project/ML/BCICIV_2b_gdf/B0904E.gdf...
GDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 466802  =      0.000 ...  1867.208 secs...


/Users/simonfei/opt/anaconda3/lib/python3.7/site-packages/mne/io/edf/edf.py:1000: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  etmode = np.fromstring(etmode, np.uint8).tolist()[0]


Extracting EDF parameters from /Users/simonfei/Motor-Imagery-App-Project/ML/BCICIV_2b_gdf/B0905E.gdf...
GDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 458570  =      0.000 ...  1834.280 secs...


/Users/simonfei/opt/anaconda3/lib/python3.7/site-packages/mne/io/edf/edf.py:1000: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  etmode = np.fromstring(etmode, np.uint8).tolist()[0]


In [4]:
events_train = [i['EVENT'] for i in headers_train]
events_test = [i['EVENT'] for i in headers_test]

stims_train = []
for event in events_train:
    s = []
    for e in event:
        if e['TYP'] == '0x0301' or e['TYP'] == '0x0302':
            s.append(e)
    stims_train.append(s)
    
stims_test = []
for event in events_test:
    s = []
    for e in event:
        #if e['TYP'] == '0x0301' or e['TYP'] == '0x0302':
        if e['TYP'] == '0x030f': #unknown cue
            s.append(e)
    stims_test.append(s)
    
stim_times_train = []
for stims in stims_train:
    st = []
    for s in stims:
        st.append(round(s['POS'] * 1000))
    stim_times_train.append(st)

stim_times_test = []
for stims in stims_test:
    st = []
    for s in stims:
        st.append(round(s['POS'] * 1000))
    stim_times_test.append(st)
    
fbe_train = []
for df in dfs_train:
    fbe = []
    for i in range(len(df)):
        fbe.append(0)
    fbe_train.append(fbe)

for i in range(len(stim_times_train)):
    fbe = fbe_train[i]
    sts = stim_times_train[i]
    for st in sts:
        fbe[round(st/4)] = 1
        
fbe_test = []
for df in dfs_test:
    fbe = []
    for i in range(len(df)):
        fbe.append(0)
    fbe_test.append(fbe)

for i in range(len(stim_times_test)):
    fbe = fbe_test[i]
    sts = stim_times_test[i]
    for st in sts:
        fbe[round(st/4)] = 1

for i in range(len(fbe_train)):
    df = dfs_train[i]
    fbe = fbe_train[i]
    df['EventStart'] = fbe

for i in range(len(fbe_test)):
    df = dfs_test[i]
    fbe = fbe_test[i]
    df['EventStart'] = fbe

In [5]:
sample_data_raw_csvs_train = [f[-10:-4] + '.csv' for f in sample_data_raw_files_train]
sample_data_raw_csvs_test = [f[-10:-4] + '.csv' for f in sample_data_raw_files_test]
for i in range(len(sample_data_raw_csvs_train)):
    f = sample_data_raw_csvs_train[i]
    dfs_train[i].to_csv('data/train/' + f, index = False)
for i in range(len(sample_data_raw_csvs_test)):
    f = sample_data_raw_csvs_test[i]
    dfs_test[i].to_csv('data/test/' + f, index = False)
labels_folder = 'data/y'
label_files = os.listdir(labels_folder)
label_files = [i for i in label_files if '.csv' in i]
label_files = ['data/y/' + i for i in label_files]
label_files_train = [i for i in label_files if 'T' in i]
label_files_test = [i for i in label_files if 'E' in i]
label_files_train.sort()
label_files_test.sort()
labels_train = [pd.read_csv(f, header = None) for f in label_files_train]
labels_test = [pd.read_csv(f, header = None) for f in label_files_test]
labels_train = [i[:120] for i in labels_train]
labels_test = [i[:120] for i in labels_test]
labels_train = pd.concat(labels_train)
labels_test = pd.concat(labels_test)
labels_train.to_csv('data/TrainLabels.csv', index = False, header = None)
labels_test.to_csv('data/true_labels.csv', index = False, header = None)

# epoching

In [6]:
train_list_arr = np.array(sorted(listdir('data/train')))
total_training_participant = 9
trial_per_subj = 3 # 9 x 3 = 27 csv files in total
train_list_arr = train_list_arr[1:]
train_list_np = np.reshape(train_list_arr, (total_training_participant,trial_per_subj))
test_list_arr = np.array(sorted(listdir('data/test')))
total_testing_participant = 9
test_trial_per_subj = 2
test_list_arr = test_list_arr[1:]
test_list_np = np.reshape(test_list_arr, (total_testing_participant,test_trial_per_subj))

In [7]:
stimulus_per_subj = 360
stimulus_per_subj_test = 240
channels = ['C3','Cz','C4']
epoch_s = 0
epoch_e = 4000 #4 seconds
fs = 250
epoch_len = int((abs(epoch_s) + abs(epoch_e)) * (fs / 1000))
train_data_list = np.empty((0, stimulus_per_subj, len(channels), epoch_len), float)
test_data_list = np.empty((0, stimulus_per_subj_test, len(channels), epoch_len), float)

In [8]:
def butter_bandpass_filter(data, lowcut, highcut, fs, order = 2):
    return data

In [10]:
bl_s = -400
bl_e = -300
for training_participant_id in range(total_training_participant):
    subject_dir_list = train_list_np[training_participant_id]
    subject_epoch = np.empty((0, len(channels), epoch_len), float)
    for trial_id in range(trial_per_subj):
        subject_dir = subject_dir_list[trial_id]
        data = generate_epoch(file_path = 'data/train/'+subject_dir, channels = channels, \
                              eeg_filter = butter_bandpass_filter, fs = fs, epoch_s = epoch_s, \
                              epoch_e = epoch_e, bl_s = bl_s, bl_e = bl_e)
        subject_epoch = np.vstack((subject_epoch, data))
    subject_epoch = np.reshape(subject_epoch, (1, stimulus_per_subj, len(channels), epoch_len))
    train_data_list = np.vstack((train_data_list, subject_epoch))

print('Epoched training data shape: '+ str(train_data_list.shape))

for testing_participant_id in range(total_testing_participant):
    subject_dir_list = test_list_np[testing_participant_id]
    subject_epoch = np.empty((0, len(channels), epoch_len), float)
    for trial_id in range(test_trial_per_subj):
        subject_dir = subject_dir_list[trial_id]
        data = generate_epoch(file_path = 'data/test/'+subject_dir, channels = channels, \
                              eeg_filter = butter_bandpass_filter, fs = fs, epoch_s = epoch_s, \
                              epoch_e = epoch_e, bl_s = bl_s, bl_e = bl_e)
        subject_epoch = np.vstack((subject_epoch, data))
    subject_epoch = np.reshape(subject_epoch, (1, stimulus_per_subj_test, len(channels), epoch_len))
    test_data_list = np.vstack((test_data_list, subject_epoch))

print('Epoched testing data shape: '+ str(test_data_list.shape))
y = pd.read_csv('data/TrainLabels.csv', header = None)
y = y.values.reshape(3240,)
print(y.shape)
yT = pd.read_csv('data/true_labels.csv', header = None) # put your train label path here
yT = yT.values.reshape(2160,)
print(yT.shape)

Epoched training data shape: (9, 360, 3, 1000)
Epoched testing data shape: (9, 240, 3, 1000)
(3240,)
(2160,)


# generateModel(X, Y)

In [18]:
X_train = train_data_list[0]
X_test = test_data_list[0]
Y_train = y[:360]
Y_test = yT[:240]

In [19]:
class myModel:
    def __init__(self):
        pass
    def fit(X, Y):
        pass
    def predict(X):
        pass
    
class XDawnLRModel(myModel): # XDAWN Covariance Preprocessing + Linear Regression Classifier
    def __init__(self):
        super().__init__()
        self.XC = XdawnCovariances(nfilter = 1) # the number of filters can be changed
        self.logreg = LogisticRegression()
        
    def fit(self, X, Y):
        X_transformed = self.XC.fit_transform(X, Y)
        X_transformed = TangentSpace(metric='riemann').fit_transform(X_transformed)
        self.logreg.fit(X_transformed,Y)
        
    def predict(self, X):
        X_transformed = self.XC.transform(X)
        X_transformed = TangentSpace(metric='riemann').fit_transform(X_transformed)
        return self.logreg.predict(X_transformed)

model = XDawnLRModel()
model.fit(X_train, Y_train)

In [20]:
Y_pred = model.predict(X_test)
print(classification_report(Y_test, Y_pred))

              precision    recall  f1-score   support

           1       0.53      0.50      0.52       120
           2       0.53      0.56      0.54       120

    accuracy                           0.53       240
   macro avg       0.53      0.53      0.53       240
weighted avg       0.53      0.53      0.53       240



In [21]:
def getPowerRatio(eeg_data, binning, eeg_fs=250):
    power, power_ratio = pyeeg.bin_power(eeg_data, binning, eeg_fs)
    return np.array(power_ratio)
def getIntervals(binning): 
    intervals = list()
    for i, val in enumerate(binning[:-1]): 
        intervals.append((val, binning[i+1]))
    return intervals

In [23]:
sub_binning=[0.5, 4, 7, 12, 30]
sub_intervals = getIntervals(sub_binning)
power_ratio1=[getPowerRatio(data[:], sub_binning) for data in X_train[:,0,:]]
power_ratio2=[getPowerRatio(data[:], sub_binning) for data in X_train[:,1,:]]
power_ratio3=[getPowerRatio(data[:], sub_binning) for data in X_train[:,2,:]]

power_ratiot1=[getPowerRatio(data[:], sub_binning) for data in X_test[:,0,:]]
power_ratiot2=[getPowerRatio(data[:], sub_binning) for data in X_test[:,1,:]]
power_ratiot3=[getPowerRatio(data[:], sub_binning) for data in X_test[:,2,:]]
power_ratio = []
for i,j,k in zip(power_ratio1,power_ratio2,power_ratio3):
    power_ratio.append(np.concatenate([i,j,k], axis = 0))
    
power_ratiot = []
for i,j,k in zip(power_ratiot1,power_ratiot2,power_ratiot3):
    power_ratiot.append(np.concatenate([i,j,k], axis = 0))

In [24]:
model = LinearDiscriminantAnalysis()
model.fit(power_ratio, Y_train)
Y_pred = model.predict(power_ratiot)
print(classification_report(Y_test, Y_pred))

              precision    recall  f1-score   support

           1       0.61      0.53      0.57       120
           2       0.59      0.66      0.62       120

    accuracy                           0.60       240
   macro avg       0.60      0.60      0.59       240
weighted avg       0.60      0.60      0.59       240

